## Library import 

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import sys
from urllib.error import HTTPError

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# ----------------------------------------------------------------------------
# PART 1 : GET ID WEBSCRAPPE
# ----------------------------------------------------------------------------

## Team Names for one season

In [2]:
import os
import datetime
os.getcwd()

'/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-data-engineering-pipeline'

In [3]:
datetime.datetime.today().strftime('%Y-%m-%d')

'2022-09-07'

In [4]:
# test = pd.read_csv('./pipeline_output/backup_dataset/schedules_training_dataset_2022-01-19 17:19:38.098995.csv')
# # test.head()

# Name proess to get player game data statistitque

In [5]:
list_players = pd.read_csv('pipeline_output/nba_list_players_multi_season_dataset_2022-08-31.csv')
list_players = list_players[['id_season', 'tm', 'Name']]
list_players.head()

# the if we want to get is:
# Lebron James
# players/j/jamesle01/gamelog/2008

,id_season,tm,Name
0,2001,ATL,Cal Bowdler
1,2001,ATL,Ira Bowman
2,2001,ATL,Sean Colson
3,2001,ATL,Chris Crawford
4,2001,ATL,Dion Glover


In [6]:
#-------------------------------
# Remove ponctuation to the Name 

list_players['Name'] = list_players['Name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# list_players[list_players['last_name_final'].str.contains('é', regex=False)]


In [7]:
#-------------------------------
# Remove specifc character 

list_players['Name'] = list_players['Name'].str.replace("'", '', regex=True)
list_players['Name'] = list_players['Name'].str.replace("-", '', regex=True)
list_players['Name'] = list_players['Name'].str.replace(".", '', regex=True)

In [8]:
#----------------------
# str split to get name and surnmae

test = list_players['Name'].str.split(pat = ' ', expand = True)
list_players[['first_name', 'last_name', 'last_name2', 'last_name3']] = test
list_players.head()

,id_season,tm,Name,first_name,last_name,last_name2,last_name3
0,2001,ATL,Cal Bowdler,Cal,Bowdler,None,None
1,2001,ATL,Ira Bowman,Ira,Bowman,None,None
2,2001,ATL,Sean Colson,Sean,Colson,None,None
3,2001,ATL,Chris Crawford,Chris,Crawford,None,None
4,2001,ATL,Dion Glover,Dion,Glover,None,None


In [9]:
#----------------------
# str split to get name and surnmae
list_players = list_players.fillna('')
list_players['last_name_final'] = list_players['last_name'] + list_players[ 'last_name2'] + list_players[ 'last_name3']
list_players.head()

,id_season,tm,Name,first_name,last_name,last_name2,last_name3,last_name_final
0,2001,ATL,Cal Bowdler,Cal,Bowdler,,,Bowdler
1,2001,ATL,Ira Bowman,Ira,Bowman,,,Bowman
2,2001,ATL,Sean Colson,Sean,Colson,,,Colson
3,2001,ATL,Chris Crawford,Chris,Crawford,,,Crawford
4,2001,ATL,Dion Glover,Dion,Glover,,,Glover


In [10]:
# list_player$Surname <- ifelse(
#   substr(list_player$Surname,
#          nchar(list_player$Surname)-3,
#          nchar(list_player$Surname)) == "(TW)",
#   substr(list_player$Surname,1, nchar(list_player$Surname)-6),
#   list_player$Surname)

# list_players['last_name_final'].str[-4:].head()
# list_players[list_players['last_name_final'].str[-4:]=='(TW)'].head()



In [11]:
# #---------------------------
# # STR replace
# # list_player$Name <- gsub("'","",list_player$Name)
# # list_player$Surname <- gsub("'","",list_player$Surname)

# # list_player$Surname <- gsub("-","",list_player$Surname)
# # list_player$Name <- gsub("-","",list_player$Name)

# # list_player$Surname <- gsub("[.]","",list_player$Surname)
# # list_player$Name <- gsub("[.]","",list_player$Name)

# list_players['last_name_final'] = list_players['last_name_final'].str.replace("'", '', regex=True)
# list_players['first_name'] = list_players['first_name'].str.replace("'", '', regex=True)

# list_players['last_name_final'] = list_players['last_name_final'].str.replace("-", '', regex=True)
# list_players['first_name'] = list_players['first_name'].str.replace("-", '', regex=True)

# list_players['last_name_final'] = list_players['last_name_final'].str.replace(".", '', regex=True)
# list_players['first_name'] = list_players['first_name'].str.replace(".", '', regex=True)

In [12]:
# Charlie Brown Jr.	b/brownch01
# 10043	Javonte Green	g/greenja01
# 10586	Justin Wright-Foreman

list_players[list_players['Name'] =='Charlie Brown Jr.']

,id_season,tm,Name,first_name,last_name,last_name2,last_name3,last_name_final


In [13]:
#----------------------
# substr(tolower(list_player$Surname),1,1)

list_players['id_part_1'] = list_players['last_name_final'].str.lower().str[0]
list_players.head()

,id_season,tm,Name,first_name,last_name,last_name2,last_name3,last_name_final,id_part_1
0,2001,ATL,Cal Bowdler,Cal,Bowdler,,,Bowdler,b
1,2001,ATL,Ira Bowman,Ira,Bowman,,,Bowman,b
2,2001,ATL,Sean Colson,Sean,Colson,,,Colson,c
3,2001,ATL,Chris Crawford,Chris,Crawford,,,Crawford,c
4,2001,ATL,Dion Glover,Dion,Glover,,,Glover,g


In [14]:
#----------------------

list_players["id_part_2"] = np.where(
    list_players["last_name"].str.lower().str.len()<5,
    list_players["last_name"],list_players['last_name'].str.lower().str[:5])

# First two characters
list_players["id_part_3"] = list_players['first_name'].str.lower().str[:2]

list_players["id_part_4"] = '01'

list_players.head()


,id_season,tm,Name,first_name,last_name,last_name2,last_name3,last_name_final,id_part_1,id_part_2,id_part_3,id_part_4
0,2001,ATL,Cal Bowdler,Cal,Bowdler,,,Bowdler,b,bowdl,ca,01
1,2001,ATL,Ira Bowman,Ira,Bowman,,,Bowman,b,bowma,ir,01
2,2001,ATL,Sean Colson,Sean,Colson,,,Colson,c,colso,se,01
3,2001,ATL,Chris Crawford,Chris,Crawford,,,Crawford,c,crawf,ch,01
4,2001,ATL,Dion Glover,Dion,Glover,,,Glover,g,glove,di,01


In [15]:
#----------------------------------
# Final id webscrapp creation

list_players['id_webscrapping'] = list_players['id_part_1'] + '/' + list_players['id_part_2'] + list_players['id_part_3'] + list_players["id_part_4"]

In [16]:
list_players.head()

,id_season,tm,Name,first_name,last_name,last_name2,last_name3,last_name_final,id_part_1,id_part_2,id_part_3,id_part_4,id_webscrapping
0,2001,ATL,Cal Bowdler,Cal,Bowdler,,,Bowdler,b,bowdl,ca,01,b/bowdlca01
1,2001,ATL,Ira Bowman,Ira,Bowman,,,Bowman,b,bowma,ir,01,b/bowmair01
2,2001,ATL,Sean Colson,Sean,Colson,,,Colson,c,colso,se,01,c/colsose01
3,2001,ATL,Chris Crawford,Chris,Crawford,,,Crawford,c,crawf,ch,01,c/crawfch01
4,2001,ATL,Dion Glover,Dion,Glover,,,Glover,g,glove,di,01,g/glovedi01


In [17]:
list_players[['id_season', 'Name',  'id_webscrapping']].head()

,id_season,Name,id_webscrapping
0,2001,Cal Bowdler,b/bowdlca01
1,2001,Ira Bowman,b/bowmair01
2,2001,Sean Colson,c/colsose01
3,2001,Chris Crawford,c/crawfch01
4,2001,Dion Glover,g/glovedi01


In [18]:
list_players.shape

(11952, 13)

## Recode manually the duplictes id due to similar names

In [19]:
# list_players['Name'] =  np.where(list_players['Name']=="James Michael","James Michael McAdoo",list_players['Name'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Walt Williams","w/williwa02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Alton Ford","f/fordal02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Cherokee Parks","p/parksch02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Rod Strickland","s/stricro02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Nick Van","v/vanexni01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Keith Van","v/vanhoke01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="James Thomas","t/thomaja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="John Thomas","t/thomajo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Donta Smith","s/smithdo04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Corliss Williamson","w/willico02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Clifford Robinson","r/robincl02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Bracey Wright","w/wrighbr02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Alvin Williams","w/willial02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Dajuan Wagner","w/wagneda02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Kevin Willis","w/willike02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Marcus Vinicius","v/vincima01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Antoine Walker","w/walkean02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Juan Carlos","n/navarju01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Bobby Jones","j/jonesbo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Mouhamed Sene","s/senesa01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Lorenzen Wright","w/wrighlo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Wally Szczerbiak","s/szczewa02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Anthony Roberson","r/roberan02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Alando Tucker","t/tuckeal02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Didier IlungaMbenga","m/mbengdj01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="David Andersen","a/anderda03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Mario West","w/westma02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Joe Smith","s/smithjo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jason Collier","c/collija02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jarron Collins","c/collija03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jermaine Taylor","t/tayloje02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jay Williams","w/willija03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jason Williams","w/willija02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jawad Williams","w/willija04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Antonio Lang","l/langan02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Brevin Knight","k/knighbr01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Gerard King","k/kingge02/",list_players['id_webscrapping'])
# list_players['id_webscrapping'] =  np.where(list_players['Name']=="Chris Johnson" & list_players['Pos=="C" ,"j/johnsch03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Mark Jones","j/jonesma03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Eddie Jones","j/jonesed02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Dwayne Jones","j/jonesdw02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Charles Jones","j/jonesch03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Ken Johnson","j/johnske03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Larry Johnson","j/johnsla02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Ervin Johnson","j/johnser02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="DerMarr Johnson","j/johnsde03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="DeMarco Johnson","j/johnsde02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Anthony Johnson","j/johnsan02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Marc Jackson","j/jacksma02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Luke Jackson","j/jackslu02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Cedric Henderson","h/hendece02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Shelden Williams","w/willish02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Todd Fuller","f/fulleto02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Patrick Ewing","e/ewingpa02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Chris Dudley","d/dudlech02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jordan Williams","w/willijo03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Mark Davis","d/davisma02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Armon Johnson","j/johnsar02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Mitchell Butler","b/butlemi02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Randy Brown","b/brownra02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Antonio Anderson","a/anderan02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Courtney Alexander","a/alexaco02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Josh Davis","d/davisjo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Keith Benson","b/bensoke02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Walker Russell","r/russewa02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Mychel Thompson","t/thompmy02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Devin Brown","b/brownde02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Dee Brown","b/brownde03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Derrick Brown","b/brownde04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Sasha Pavlovic","p/pavloal01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Damien Wilkins","w/wilkida02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="James White","w/whiteja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Dominique Jones","j/jonesdo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Daequan Cook","c/cookda02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Martell Webster","w/webstma02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Diante Garrett","g/garredi02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Nando De","d/decolna01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Rodney White","w/whitero02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Royce White","w/whitero03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Darius JohnsonOdom","j/johnsda03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Chris Smith","s/smithch05/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Rashard Lewis","l/lewisra02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Ray Allen","a/allenra02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Stephen Jackson","j/jacksst02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Tony Mitchell","m/mitchto02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Ronnie Brewer","b/brewero02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jason Collins","c/collija04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Glen Rice","r/ricegl01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Larry Drew","d/drewla02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Henry Walker","w/walkebi01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Shawne Williams","w/willish03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jeffery Taylor","t/tayloje03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Chris Johnson","j/johnsch04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jerian Grant","g/grantje02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jordan Hamilton","h/hamiljo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Greg Smith","s/smithgr02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Andre Miller","m/millean02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Kevin Martin","m/martike02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jeff Ayres","p/pendeje02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Josh Smith","s/smithjo03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jason Thompson","t/thompja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Mo Williams","w/willima01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="PJ Hairston","h/hairspj02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Sheldon Mac","m/mcclesh01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="David Lee","l/leeda02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Gerald Henderson","h/hendege02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Reggie Williams","w/willire02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="John Lucas","l/lucasjo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Willie Reed","r/reedwi02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Brandan Wright","w/wrighbr03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Metta World","a/artesro01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Brice Johnson","j/johnsbr02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Bobby Brown","b/brownbo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="James Michael McAdoo","m/mcadoja01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Matt Barnes","b/barnema02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="James Jones","j/jonesja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Dahntay Jones","j/jonesda02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Bojan Bogdanovic","b/bogdabo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Derrick Williams","w/willide02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Edy Tavares","t/tavarwa01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Mike Dunleavy","d/dunlemi02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Taurean Prince","p/princta02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Isaiah Thomas","t/thomais02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Damion Lee","l/leeda03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Marcus Morris","m/morrima03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Markieff Morris","m/morrima02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jaylen Brown","b/brownja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jabari Bird","b/birdja01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Kemba Walker","w/walkeke02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Marvin Williams","w/willima02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Larry Nance","n/nancela02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jeff Green","g/greenje02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Cedi Osman","o/osmande01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="John Holland","h/hollajo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Wesley Matthews","m/matthwe02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Harrison Barnes","b/barneha02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Dennis Smith","s/smithde03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jalen Jones","j/jonesja04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="JJ Barea","b/bareajo01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Maxi Kleber","k/klebima01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Stanley Johnson","j/johnsst04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Damian Jones","j/jonesda03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Clint Capela","c/capelca01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Nene NA","h/hilarne01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Markel Brown","b/brownma02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Joe Johnson","j/johnsjo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Luc Mbah","m/mbahalu01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Glenn Robinson","r/robingl01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Tobias Harris","h/harrito02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Lou Williams","w/willilo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Gary Payton","p/paytoga01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Derrick Jones","j/jonesde02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Sterling Brown","b/brownst02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Xavier Munford","m/munfoxa02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Marshall Plumlee","p/plumlma02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Anthony Brown","b/brownan02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Anthony Davis","d/davisan02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jordan Crawford","c/crawfjo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Tim Hardaway","h/hardati02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Troy Williams","w/willitr02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Frank Ntilikina","n/ntilila01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Andre Roberson","r/roberan03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Dakari Johnson","j/johnsda04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Daniel Hamilton","h/hamilda02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jonathon Simmons","s/simmojo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Brandon Knight","k/knighbr03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Josh Jackson","j/jacksjo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Alan Williams","w/willial03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="JaKarr Sampson","s/sampsja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Danny Green","g/greenda02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Patty Mills","m/millspa02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jason Smith","s/smithja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Charlie Brown","b/brownch02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Robert Williams","w/williro04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Caleb Martin","m/martica02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Cristiano Felício","m/feliccr01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Louis King","k/kinglo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Alize Johnson","j/johnsal02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Kevin Porter","p/porteke02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Ante Zizic","z/zizican01/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Josh Reaves","r/reavejo02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Brian Bowen","b/bowenbr02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jaren Jackson","j/jacksja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Kelan Martin","m/martike03/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jacob Evans","e/evansja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Jaxson Hayes","h/hayesja02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Kenrich Williams","w/willike04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Deonte Burton","b/burtode02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Johnathan Williams","w/willijo04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Matt Thomas","t/thomama02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Glenn Robinson","r/robingl02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Cameron Johnson","j/johnsca02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Gary Trent","t/trentga02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Keldon Johnson","j/johnske04/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Terence Davis","d/daviste02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Justin WrightForeman","w/wrighju02/",list_players['id_webscrapping'])
list_players['id_webscrapping'] =  np.where(list_players['Name']=="Miles Bridges","b/bridgmi02/",list_players['id_webscrapping'])



#######

# 324	James Robinson	r/robinja01
# 484	Mike Smith	s/smithmi01
# 780	Donny Marshall	m/marshdo01
# 3329	Steven Smith	s/smithst01
# 5577	JaJuan Johnson	j/johnsja01
# 5754	Trey Thompkins	t/thomptr01
# 8681	Derrick Jones Jr.	j/jonesde01
# 9123	Matt Williams	w/willima01
# 9771	Dairis Bertans	b/bertada01
# 10019	Charlie Brown Jr.	b/brownch01
# 10043	Javonte Green	g/greenja01
# 10586	Justin Wright-Foreman	w/wrighju01
# 10833	Kenyon Martin Jr.	m/martike01
# 10977	Jaden McDaniels	m/mcdanja01
# 11115	Jalen Smith	s/smithja01
# 11238	Chaundee Brown Jr.	b/brownch01
# 11249	Jalen Johnson	j/johnsja01
# 11298	Kessler Edwards	e/edwarke01
# 11478	Jalen Green	g/greenja01
# 11589	Ziaire Williams	w/willizi01
# 11675	Trey Murphy III	m/murphtr01
# 11723	Jeremiah Robinson-Earl	r/robinje01
# 11888	David Johnson	j/johnsda01
# 11902	Jared Butler	b/butleja01

In [20]:
list_players_unique_name = list_players[['Name', 'id_webscrapping']].drop_duplicates(subset=['Name'])
print(list_players_unique_name[list_players_unique_name.duplicated(subset=['id_webscrapping'])].shape)
list_players_unique_name[list_players_unique_name.duplicated(subset=['id_webscrapping'])]

(22, 2)


,Name,id_webscrapping
324,James Robinson,r/robinja01
484,Mike Smith,s/smithmi01
780,Donny Marshall,m/marshdo01
3329,Steven Smith,s/smithst01
5577,JaJuan Johnson,j/johnsja01
5754,Trey Thompkins,t/thomptr01
8681,Derrick Jones Jr,j/jonesde01
9123,Matt Williams,w/willima01
9771,Dairis Bertans,b/bertada01
10019,Charlie Brown Jr,b/brownch01


In [21]:
final = list_players[['id_season', 'Name', 'id_webscrapping']]
final.head()

,id_season,Name,id_webscrapping
0,2001,Cal Bowdler,b/bowdlca01
1,2001,Ira Bowman,b/bowmair01
2,2001,Sean Colson,c/colsose01
3,2001,Chris Crawford,c/crawfch01
4,2001,Dion Glover,g/glovedi01


In [23]:
final.sort_values(by=['Name', 'id_season']).head()

,id_season,Name,id_webscrapping
236,2001,AC Green,g/greenac01
64,2001,AJ Guyton,g/guytoaj01
546,2002,AJ Guyton,g/guytoaj01
1086,2003,AJ Guyton,g/guytoaj01
8374,2017,AJ Hammons,h/hammoaj01


In [34]:
test = final.drop_duplicates(subset=['Name'])
test = test[test.duplicated(['id_webscrapping'])]
print(test.shape)
test.sort_values(by=['Name', 'id_season']).head(100)

(22, 3)


,id_season,Name,id_webscrapping
10019,2020,Charlie Brown Jr,b/brownch01
11238,2022,Chaundee Brown Jr,b/brownch01
9771,2019,Dairis Bertans,b/bertada01
8681,2017,Derrick Jones Jr,j/jonesde01
780,2002,Donny Marshall,m/marshdo01
5577,2012,JaJuan Johnson,j/johnsja01
10977,2021,Jaden McDaniels,m/mcdanja01
11478,2022,Jalen Green,g/greenja01
11249,2022,Jalen Johnson,j/johnsja01
11115,2021,Jalen Smith,s/smithja01


# ----------------------------------------------------------------------------
# PART 2: WEBSCRAPPE USING ID
# ----------------------------------------------------------------------------

In [36]:
nba_id_webscrappe_players = pd.read_csv('pipeline_output/nba_id_webscrappe_players_2022-09-07.csv')

In [37]:
nba_id_webscrappe_players.head()

,id_season,Name,id_webscrapping
0,2001,AC Green,g/greenac01
1,2001,AJ Guyton,g/guytoaj01
2,2002,AJ Guyton,g/guytoaj01
3,2003,AJ Guyton,g/guytoaj01
4,2017,AJ Hammons,h/hammoaj01


In [80]:
nba_id_webscrappe_players.shape

(11952, 3)

In [83]:
nba_id_webscrappe_players.id_season.astype(str)

0        2001
1        2001
2        2002
3        2003
4        2017
5        2010
6        2011
7        2012
8        2013
9        2014
10       2015
11       2015
12       2015
13       2008
14       2009
15       2010
16       2011
17       2011
18       2013
19       2013
20       2014
21       2014
22       2015
23       2016
24       2017
25       2018
26       2015
27       2016
28       2017
29       2018
30       2019
31       2020
32       2021
33       2021
34       2022
35       2008
36       2009
37       2010
38       2010
39       2011
40       2012
41       2013
42       2014
43       2014
44       2016
45       2017
46       2018
47       2022
48       2019
49       2020
50       2021
51       2022
52       2022
53       2018
54       2001
55       2002
56       2003
57       2004
58       2005
59       2006
60       2007
61       2006
62       2021
63       2022
64       2022
65       2001
66       2002
67       2003
68       2004
69       2005
70       2005
71    

In [81]:
#------------------------------------------------------

test = nba_id_webscrappe_players.head(50)
players = pd.DataFrame()
players_to_add_manually = pd.DataFrame()
#------------------------------------------------------
# For Loop Throught all the team abrev for the given season

for index, row in test.iterrows():
    # URL to scrape
    player_id = row['id_webscrapping']
    id_season = row['id_season'].astype(str)
    url = f'https://www.basketball-reference.com/players/{player_id}/gamelog/{id_season}'

    # collect HTML data and create beautiful soup object:
    # collect HTML data
    try:
        html = urlopen(url)
        
        # create beautiful soup object from HTML
        soup = BeautifulSoup(html, "html.parser" )

        try:
            rows = soup.findAll('table')[-1]
            rows = rows.find_all('tr')

            rows_data = [[td.getText() for td in rows[i].findAll('td')]
                                for i in range(len(rows))]

            if len(rows_data) != 0:
                # create the dataframe
                players_tmp = pd.DataFrame(rows_data)
                players_tmp = players_tmp.reset_index()

                cols = [
                    'rk', 'game_id', 'game_date',
                    'age',
                    'tm','ext_dom', 'ppp','results',
                    'gs', 'mp',
                    'fg', 'fga', 'fg_prct%', '3p', '3pa', '3p_prct', 'ft',	'fta',	'ft_prct', 
                    'orb',	'drb',	'trb',	
                    'ast',	
                    'stl',	'blk',	'tov',	'pf',	
                    'pts',	'player_game_score', 'plus_minus']

                players_tmp.columns =  cols
                players_tmp = players_tmp.dropna()
                players_tmp['id_season'] = id_season

                players = players.append(players_tmp)

        except:
            players_to_add_manually = players_to_add_manually.append(row)
            print("An exception occurred")

    except HTTPError as http_error:
        # print(row)
        players_to_add_manually = players_to_add_manually.append(row)
        continue

players = players[[
    'id_season',
    'rk', 'game_id', 'game_date',
    'age',
    'tm','ext_dom', 'ppp','results',
    'gs', 'mp',
    'fg', 'fga', 'fg_prct%', '3p', '3pa', '3p_prct', 'ft',	'fta',	'ft_prct', 
    'orb',	'drb',	'trb',	
    'ast',	
    'stl',	'blk',	'tov',	'pf',	
    'pts',	'player_game_score', 'plus_minus']]



An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

In [82]:
players_to_add_manually

,id_season,Name,id_webscrapping
227,2022.0,Aleem Ford,f/fordal01
1277,2022.0,Brandon Williams,w/willibr01
1558,2022.0,Cam Thomas,t/thomaca01
1599,2022.0,Carlik Jones,j/jonesca01
1600,2022.0,Carlik Jones,j/jonesca01
1756,2002.0,Charles Smith,s/smithch01
1757,2003.0,Charles Smith,s/smithch01
1758,2006.0,Charles Smith,s/smithch01
1759,2006.0,Charles Smith,s/smithch01
1768,2020.0,Charlie Brown Jr,b/brownch01


In [ ]:
# Webscrapping test::

url = 'https://www.basketball-reference.com/players/d/divindo01/gamelog/2020'

In [122]:
list_players[list_players['id_webscrapping'] =='m/millean01']


,id_season,tm,Name,first_name,last_name,last_name2,last_name3,last_name_final,id_part_1,id_part_2,id_part_3,id_part_4,id_webscrapping
13,2001,ATL,Anthony Miller,Anthony,Miller,,,Miller,m,mille,an,01,m/millean01
85,2001,CLE,Andre Miller,Andre,Miller,,,Miller,m,mille,an,01,m/millean01
176,2001,HOU,Anthony Miller,Anthony,Miller,,,Miller,m,mille,an,01,m/millean01
338,2001,PHI,Anthony Miller,Anthony,Miller,,,Miller,m,mille,an,01,m/millean01
566,2002,CLE,Andre Miller,Andre,Miller,,,Miller,m,mille,an,01,m/millean01
1132,2003,LAC,Andre Miller,Andre,Miller,,,Miller,m,mille,an,01,m/millean01
1524,2004,DEN,Andre Miller,Andre,Miller,,,Miller,m,mille,an,01,m/millean01
1946,2005,ATL,Anthony Miller,Anthony,Miller,,,Miller,m,mille,an,01,m/millean01
2044,2005,DEN,Andre Miller,Andre,Miller,,,Miller,m,mille,an,01,m/millean01
2573,2006,DEN,Andre Miller,Andre,Miller,,,Miller,m,mille,an,01,m/millean01


In [142]:
list_players[list_players['Name'] =='Ty Lawson']

,id_season,tm,Name,first_name,last_name,last_name2,last_name3,last_name_final,id_part_1,id_part_2,id_part_3,id_part_4,id_webscrapping
4615,2010,DEN,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
5135,2011,DEN,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
5666,2012,DEN,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
6207,2013,DEN,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
6734,2014,DEN,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
7284,2015,DEN,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
7900,2016,HOU,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
7914,2016,IND,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
8718,2017,SAC,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01
9383,2018,WAS,Ty Lawson,Ty,Lawson,,,Lawson,l,lawso,ty,01,l/lawsoty01


# ----------------------------------------------------------------------------
# PART 3: GET Team salary
# ----------------------------------------------------------------------------

In [13]:
salaries_url = 'https://www.basketball-reference.com/teams/MIA/2003.html'
salaries_response = requests.get(salaries_url)
page = salaries_response.text

soup = BeautifulSoup(page)
print(soup)

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://cdn.ssref.net/req/202209081" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '__tcfapiLo

In [41]:

url = 'https://www.basketball-reference.com/teams/MIA/2003.html'
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser" )
#rows = soup.findAll('tr')
#rows = rows.find_all('tr')
soup



<!DOCTYPE html>

<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://cdn.ssref.net/req/202209081" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '__tcfapiL

In [32]:
# team = row['team_abrev']
#url = f"https://www.basketball-reference.com/teams/{team}/{SEASON}.html"
url = 'https://www.basketball-reference.com/teams/MIA/2003.html'
# url = 'https://www.basketball-reference.com/players/c/curryst01.html#all_all_salaries'
print(url)

# collect HTML data and create beautiful soup object:
# collect HTML data

# soup = BeautifulSoup(soup.find("div", {"id": "all_salaries2"}))

# rows_data = [[td.getText() for td in rows[i].findAll('td')]
#                     for i in range(len(rows))]

# print(rows_data)

# if len(rows_data) != 0:
#     # create the dataframe
#     players_tmp = pd.DataFrame(rows_data)

#     cols = [
#         'Name', 
#         'Position', 
#         'Ht', 
#         'Wt', 
#         'BirthDate',
#         'Nationality',
#         'Experience', 
#         'College']

#     players_tmp.columns =  cols
#     players_tmp = players_tmp.dropna()
#     players_tmp['id_season'] = SEASON
#     players_tmp['tm'] = team

#     players = players.append(players_tmp)



https://www.basketball-reference.com/teams/MIA/2003.html


TypeError: 'NoneType' object is not callable

In [ ]:

# library(tidyr)
# list_player <- list_player %>% separate(Player, c("Name","Surname"), " ")

# list_player$Surname <- ifelse(
#   substr(list_player$Surname,
#          nchar(list_player$Surname)-3,
#          nchar(list_player$Surname)) == "(TW)",
#   substr(list_player$Surname,1, nchar(list_player$Surname)-6),
#   list_player$Surname)

# list_player$Name <- gsub("'","",list_player$Name)
# list_player$Surname <- gsub("'","",list_player$Surname)

# list_player$Surname <- gsub("-","",list_player$Surname)
# list_player$Name <- gsub("-","",list_player$Name)

# list_player$Surname <- gsub("[.]","",list_player$Surname)
# list_player$Name <- gsub("[.]","",list_player$Name)

# list_player$id_webscrappe <- paste0(substr(tolower(list_player$Surname),1,1),
#                                     "/",
#                                     ifelse(nchar(list_player$Surname)<5,
#                                            substr(tolower(list_player$Surname),1,nchar(list_player$Surname)),
#                                            substr(tolower(list_player$Surname),1,5)),
#                                     substr(tolower(list_player$Name),1,2),
#                                     "01",
#                                     "/gamelog/",
#                                     year,
#                                     "/")
